In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#랜덤시드 고정
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
#gpu 사용가능하면 사용하는 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
#데이터 로드
sample = pd.read_csv('/kaggle/input/2022aifinalexamp2/submit.csv')

In [5]:
#학습데이터, 테스트데이터 경로 지정
from os.path import join
from glob import glob

train_path = '/kaggle/input/2022aifinalexamp2/train_data/train_images'
train_image_path = glob(join(train_path,'*','*'))
test_path = '/kaggle/input/2022aifinalexamp2/test_data/test_images'
test_image_path = glob(join(test_path,'*'))
test_sorted_path = []
for i in range(len(test_image_path)):
    test_sorted_path.append(test_path + '/' + str(i) + '.png')

In [6]:
#학습데이터셋, 테스트데이터셋 정의
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class Datasets(Dataset):
    def __init__(self, path, state='',transform=None):
        self.state = state
        self.transform = transform
        self.path = path
        
        if self.state == 'train':
            self.label = [int(i.split('/')[-2]) for i in self.path]
    
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, index):
        data = self.path[index]
        data = Image.open(data).convert('RGB')
        data = self.transform(data)
        
        if self.state == 'test':
            return data
        
        else:
            return data, torch.from_numpy(np.array(self.label[index]))

In [7]:
#데이터 정규화 방식

import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [8]:
#학습데이터셋, 테스트데이터셋 설정
train_dataset = Datasets(train_image_path, state='train', transform=transform)
test_dataset = Datasets(test_sorted_path, state='test', transform=transform)

In [9]:
#학습데이터로더, 테스트데이터로더 설정
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [10]:
#사전학습된 모델을 가져와서 사용
import torchvision.models as models
import torch.nn as nn

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 10, bias=True)
nn.init.xavier_uniform_(model.fc.weight)

model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [11]:
#optimizer 정의
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#loss 정의(다중분류문제이므로 CrossEntropyLoss)
loss = nn.CrossEntropyLoss()

In [12]:
#모델학습
from tqdm import tqdm

epochs = 5
for epoch in range(epochs):
    costs = []
    for x, y in tqdm(train_dataloader):
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(x)
        cost = loss(hypothesis, y)
        costs.append(cost.item())
        cost.backward()
        optimizer.step()
    
    average = np.mean(costs)
    print(f"epoch : {epoch}, cost : {average}")

100%|██████████| 782/782 [08:17<00:00,  1.57it/s]


epoch : 0, cost : 1.0082679272383985


100%|██████████| 782/782 [06:47<00:00,  1.92it/s]


epoch : 1, cost : 0.6643673186869268


100%|██████████| 782/782 [06:42<00:00,  1.94it/s]


epoch : 2, cost : 0.5366871466722025


100%|██████████| 782/782 [06:38<00:00,  1.96it/s]


epoch : 3, cost : 0.45601733518607174


100%|██████████| 782/782 [06:23<00:00,  2.04it/s]

epoch : 4, cost : 0.360692925875068


In [13]:
#테스트데이터를 이용해서 모델 평가
model.eval()
predicts = []
with torch.no_grad():
    for x in tqdm(test_dataloader):
        x = x.to(device)
        hypothesis = model(x)
        predict = torch.argmax(hypothesis, dim=1)
        predicts.extend(predict.cpu().detach().numpy())

100%|██████████| 157/157 [00:39<00:00,  4.01it/s]


In [14]:
#결과값을 csv파일로 변환
sample['label'] = predicts
sample.to_csv('submit.csv',index=False)